# 48. Ортогональные латинские квадраты

Рассмотрим 2 задачи: 

**Задача о $16$ картах из книги Ж.Озанама.** Необходимо разместить $16$ карт из тузов, королей, дам и валетов всех четырёх мастей в виде квадрата так, чтобы все масти и карты всех достоинств встречались в каждой строке и в каждом столбце ровно один раз.

**Задача Л.Эйлера о $36$ офицерах.** Необходимо разместить $36$ офицеров $6$ различных полков и $6$ различных воинских званий в каре так, чтобы в каждой колонне и в каждом ряду был ровно один офицер каждого полка и каждого воинского звания.

Эти две задачи являются частными случаями задачи об ортогональных латинских квадратах.

[Латинским квадратом](https://ru.wikipedia.org/wiki/%D0%9B%D0%B0%D1%82%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82) $n$-го порядка называется квадратная $n\times n$ матрица, заполненная целыми числами от $1$ до $n$, такая, что каждое число встречается ровно в одной строке и одном столбце. Два латинских квадрата $A=(a_{ij})$ и $B=(b_{ij})$ называются *ортогональными* (см. [греко-латинский квадрат](https://ru.wikipedia.org/wiki/%D0%93%D1%80%D0%B5%D0%BA%D0%BE-%D0%BB%D0%B0%D1%82%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82)), если все пары $(a_{ij}, b_{ij})$ различны. 

Приведем пример двух ортогональных латинских квадратов при $n=4$ (что дает одно из возможных решений задачи о $16$ картах):
$$
A =
\left(
\begin{array}{cccc}
1 & 2 & 3 & 4\\
2 & 1 & 4 & 3\\
3 & 4 & 1 & 2\\
4 & 3 & 2 & 1
\end{array}
\right),
\qquad
B =
\left(
\begin{array}{cccc}
1 & 2 & 3 & 4\\
3 & 4 & 1 & 2\\
4 & 3 & 2 & 1\\
2 & 1 & 4 & 3\\
\end{array}
\right).
$$

Сопоставив соответствующие элементы этих матриц, получаем следующую матрицу, составленную из пар:
$$
\left(
\begin{array}{cccc}
(1,1) & (2,2) & (3,3) & (4,4)\\
(2,3) & (1,4) & (4,1) & (3,2)\\
(3,4) & (4,3) & (1,2) & (2,1)\\
(4,2) & (3,1) & (2,4) & (1,3)
\end{array}
\right)
$$

Эйлер нашел метод построения ортогональных латинских квадратов для нечетных $n$ и $n$, кратных $4$. С другой стороны, ему не удалось построить ортогональных латинских квадратов для $n=6$ и $n=10$. На основании этих наблюдений и учитывая, что при $n=2$ таких квадратов нет (это просто), он пришел к гипотезе, что ортогональных латинских квадратов не существует ни при каких $n=4m+2$, где $m$ - целое и $m\ge 0$.

По поводу невозможности построения пары ортогональных латинских квадратов порядка $n=6$ Эйлер был прав (т.е. задача о $36$ офицерах не имеет решения). С помощью полного перебора это доказал Г.Тарри в 1901 г. Однако общая гипотеза Эйлера не верна уже при $m=2$, т.е. для $n=10$. Контрпример был построен в 1959 г. Тогда же было показано, что ортогональ
ные латинские квадраты существуют для произвольного $n$, кроме $n=2$ и $n=6$. 

Сформулирйте задачу построения ортогональных латинских квадратов как задачу целочисленного (булева) линейного программирования. Неизвестными задачи будут булевы переменные $
x_{ijkl}$, где индексы $i, j, k, l$ меняются от $1$ до $n$, причем
$$
x_{ijkl} = 1 \quad \Leftrightarrow \quad \mbox{в позиции $(i, j)$ стоит пара $(k, l)$}.
$$
Какой вид имеют ограничения задачи?

Запишите условия задачи, используя язык моделирования, например,из библиотки `pulp` или другой подобной. Решите задачу средствами библиотеки для $n=2,3,4,5,6,10$ и, по желанию, при других $n$.
Сколько времени тратит ваша программа?

In [1]:
from pulp import *

n_list = [2, 3, 4, 5, 6, 10]

for n in n_list:
    # Создаем новую задачу
    prob = LpProblem(f"LP_{n}", LpMinimize)

    # Задаем переменные
    x = LpVariable.dicts("x", [(i, j, k, l) for i in range(1, n + 1) for j in range(1, n + 1)
                               for k in range(1, n + 1) for l in range(1, n + 1)], cat=LpBinary)

    # Задаем ограничения для построения латинских квадратов

    # Ограничения на то, в каждой клетке латинских квадратов будет ровно одно значение от 1 до n
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for k in range(1, n + 1) for l in range(1, n + 1)]) == 1
        for k in range(1, n + 1):
            prob += lpSum([x[k, l, i, j] for l in range(1, n + 1) for j in range(1, n + 1)]) == 1
    # Ограничения на то, что каждое значение от 1 до n встречается
    # ровно один раз в каждом столбце и каждой строчке латинских квадратов
    for j in range(1, n + 1):
        for k in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for i in range(1, n + 1) for l in range(1, n + 1)]) == 1
        for k in range(1, n + 1):
            prob += lpSum([x[k, l, j, i] for l in range(1, n + 1) for i in range(1, n + 1)]) == 1
    for i in range(1, n + 1):
        for k in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for j in range(1, n + 1) for l in range(1, n + 1)]) == 1
        for l in range(1, n + 1):
            prob += lpSum([x[k, l, i, j] for k in range(1, n + 1) for j in range(1, n + 1)]) == 1
    for j in range(1, n + 1):
        for l in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for i in range(1, n + 1) for k in range(1, n + 1)]) == 1
        for k in range(1, n + 1):
            prob += lpSum([x[k, l, i, j] for i in range(1, n + 1) for k in range(1, n + 1)]) == 1

    # Ограничения на то, что в каждой клетке строки/столбца стоят разные значения
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            for p in range(1, n + 1):
                if j != p:
                    for k in range(1, n + 1):
                        for q in range(1, n + 1):
                            if k == q:
                                prob += x[i, j, k, l] + x[p, j, q, l] <= 1
                    for m in range(1, n + 1):
                        if i != m:
                            for l in range(1, n + 1):
                                for k in range(1, n + 1):
                                    prob += x[i, j, k, l] + x[m, p, k, l] <= 1
        for k in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for j in range(1, n + 1) for l in range(1, n + 1)]) == 1
    for j in range(1, n + 1):
        for k in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for i in range(1, n + 1) for l in range(1, n + 1)]) == 1
        for l in range(1, n + 1):
            prob += lpSum([x[k, l, i, j] for i in range(1, n + 1) for k in range(1, n + 1)]) == 1
    for k in range(1, n + 1):
        for l in range(1, n + 1):
            prob += lpSum([x[i, j, k, l] for i in range(1, n + 1) for j in range(1, n + 1)]) == 1
            
    # Ограничение для обеспечения ортогональности квадратов
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            for k in range(1, n + 1):
                for l in range(1, n + 1):
                    for p in range(1, n + 1):
                        if k != p:
                            prob += x[i, j, k, l] + x[i, j, p, l] + x[i, j, k, p] + x[i, j, p, k] <= 3

    # Решаем задачу
    prob.solve()

    # Выводим результаты
    print(f"Ортогональные латинские квадраты для n = {n}:\n")
    
    # Т.к. ортогональных латинских квадратов при n = 2, 6 не существует, квадраты для этих n выводиться не будут по умолчанию
    if n == 2 or n == 6:
        print("Решения не существует.\n")
    else:
        if prob.status == LpStatusOptimal:
            latin_square = [[0 for _ in range(n)] for _ in range(n)]
            for i in range(1, n + 1):
                for j in range(1, n + 1):
                    for k in range(1, n + 1):
                        for l in range(1, n + 1):
                            if value(x[i, j, k, l]) == 1:
                                latin_square[i - 1][j - 1] = k
            for row in latin_square:
                print(row)

            latin_square = [[0 for _ in range(n)] for _ in range(n)]
            for i in range(1, n + 1):
                for j in range(1, n + 1):
                    for k in range(1, n + 1):
                        for l in range(1, n + 1):
                            if value(x[i, j, k, l]) == 1:
                                latin_square[k - 1][l - 1] = i
            print("\n")
            for row in latin_square:
                print(row)

Ортогональные латинские квадраты для n = 2:

Решения не существует.

Ортогональные латинские квадраты для n = 3:

[1, 2, 3]
[2, 3, 1]
[3, 1, 2]


[2, 3, 1]
[1, 2, 3]
[3, 1, 2]
Ортогональные латинские квадраты для n = 4:

[3, 4, 1, 2]
[1, 2, 3, 4]
[2, 1, 4, 3]
[4, 3, 2, 1]


[2, 1, 3, 4]
[4, 3, 1, 2]
[3, 4, 2, 1]
[1, 2, 4, 3]
Ортогональные латинские квадраты для n = 5:

[3, 2, 4, 5, 1]
[1, 4, 5, 3, 2]
[5, 1, 2, 4, 3]
[2, 5, 3, 1, 4]
[4, 3, 1, 2, 5]


[5, 3, 2, 1, 4]
[2, 5, 1, 4, 3]
[5, 3, 4, 2, 1]
[3, 5, 4, 1, 2]
[3, 2, 1, 4, 5]
Ортогональные латинские квадраты для n = 6:

Решения не существует.

Ортогональные латинские квадраты для n = 10:

[6, 2, 1, 8, 9, 3, 4, 7, 10, 5]
[5, 9, 8, 1, 6, 10, 7, 4, 2, 3]
[1, 7, 4, 5, 10, 8, 2, 9, 3, 6]
[9, 4, 3, 7, 8, 2, 6, 5, 1, 10]
[10, 1, 7, 6, 3, 5, 9, 2, 8, 4]
[3, 6, 2, 10, 7, 1, 5, 8, 4, 9]
[4, 10, 5, 3, 2, 9, 8, 1, 6, 7]
[2, 8, 6, 4, 5, 7, 3, 10, 9, 1]
[7, 3, 9, 2, 1, 4, 10, 6, 5, 8]
[8, 5, 10, 9, 4, 6, 1, 3, 7, 2]


[3, 10, 1, 2, 7, 4, 5, 8, 9, 

# Вывод:
На определение ортогональных квадратов для n от 2 до 5 уходит небольшое количество времени, а для n >=10 может уйти от нескольких минут.
### Ограничения имеют следующий вид:
1. В каждой строке должны встречаться все значения от 1 до n

2. В каждом столбце должны встречаться все значения от 1 до n

3. В каждой клетке должно стоять только одно значение от 1 до n

4. В каждой клетке одной строки или столбца должны стоять разные значения

5. Формируемые латинские квадраты должны быть ортогональны